
* **Adınız:** Beyza
* **Soyadınız:** Salman
* **Okul Numaranız:** 2212721049
* **GitHub Repo Bağlantısı:** https://github.com/BeyzaSalmann/MLP_Flask


In [8]:

# Kütüphaneler ve Veri Yükleme

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')

# Veriyi Yükle
df = pd.read_csv('insurance.csv')

print(f"Veri Seti Boyutu: {df.shape}")
print("\n--- Orijinal Veri (İlk 5 Satır) ---")
display(df.head())


# VERİ ÖN İŞLEME VE ENCODING
# sex (male/female), smoker (yes/no), region (sw/se/nw/ne) -> Sayıya çevriliyor

le = LabelEncoder()

# 1. Cinsiyet (sex): Kadın=0, Erkek=1
df['sex'] = le.fit_transform(df['sex'])

# 2. Sigara (smoker): Yes=1, No=0
df['smoker'] = le.fit_transform(df['smoker'])

# 3. Bölge (region): 0, 1, 2, 3 olarak kodlar
df['region'] = le.fit_transform(df['region'])

print("\n--- Encoding Sonrası Veri (Hepsi Sayısal Oldu) ---")
display(df.head())

# Veri Tiplerini Kontrol Et
print(df.info())

Veri Seti Boyutu: (1338, 7)

--- Orijinal Veri (İlk 5 Satır) ---


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520



--- Encoding Sonrası Veri (Hepsi Sayısal Oldu) ---


,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   int64  
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   int64  
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(5)
memory usage: 73.3 KB
None


In [9]:
#BACKWARD ELIMINATION (İSTATİSTİKSEL ELEME)
import statsmodels.api as sm

# 1. Bağımlı (y) ve Bağımsız (X) Değişkenleri Ayıralım
y = df['charges']  # Tahmin edeceğimiz şey (Fiyat)
X = df.drop(['charges'], axis=1) # Fiyat dışındaki her şey (Özellikler)

# 2. İstatistiksel Model İçin Sabit Ekleme (Constant)
X_list = sm.add_constant(X)

# 3. Modeli Eğitimi ve Rapor
model = sm.OLS(y, X_list).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     668.1
Date:                Mon, 15 Dec 2025   Prob (F-statistic):               0.00
Time:                        09:49:34   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1331   BIC:                         2.715e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.182e+04    955.130    -12.371      0.0

In [11]:
# ANLAMSIZ ÖZNİTELİĞİN ELENMESİ
# 'sex' sütununun p-değeri 0.694 çıktı (0.05'ten büyük).
# Bu yüzden modelden çıkarıldı.
df_final = df.drop(['sex'], axis=1)
print("'sex' sütunu başarıyla silindi (Backward Elimination Tamamlandı).")

#MODEL EĞİTİMİ (TRAIN - TEST SPLIT)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Değişkenleri Belirle
y = df_final['charges']             # Hedef
X = df_final.drop(['charges'], axis=1) # Özellikler

# 2. Veriyi Böl (%80 Eğitim, %20 Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nEğitim Verisi: {X_train.shape}")
print(f"Test Verisi: {X_test.shape}")

# 3. Modeli Eğit
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# 4. Tahmin Yap
y_pred = lr_model.predict(X_test)


# MODEL PERFORMANSI VE METRİKLER

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("\n--- FİNAL MODEL SONUÇLARI ---")
print(f"R-Kare (R2) Skoru         : {r2:.4f}")
print(f"Ortalama Mutlak Hata (MAE): {mae:.2f}")
print(f"Ortalama Kare Hata (MSE)  : {mse:.2f}")

# Yorum
if r2 > 0.70:
    print("\n Model %70 üzeri başarı yakaladı.")
else:
    print("\nModel geliştirilmeli.")

'sex' sütunu başarıyla silindi (Backward Elimination Tamamlandı).

Eğitim Verisi: (1070, 5)
Test Verisi: (268, 5)

--- FİNAL MODEL SONUÇLARI ---
R-Kare (R2) Skoru         : 0.7833
Ortalama Mutlak Hata (MAE): 4187.32
Ortalama Kare Hata (MSE)  : 33640657.14

 Model %70 üzeri başarı yakaladı.


In [13]:
# MODELİN KAYDEDİLMESİ (DEPLOYMENT)
import pickle
pickle.dump(lr_model, open('insurance_model.pkl', 'wb'))

